In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf

In [4]:
batch_size = 32
img_height = 180
img_width = 180

In [5]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '../dataset/training_set',
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

AttributeError: module 'tensorflow.keras.preprocessing' has no attribute 'image_dataset_from_directory'

In [4]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '../dataset/test_set',
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 400 files belonging to 3 classes.
Using 80 files for validation.


In [9]:
class_names = train_ds.class_names
print(class_names)

['Oblong', 'Round', 'Square']


In [10]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [11]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 180, 180, 3)
(32,)


In [2]:
from tensorflow.keras import layers

normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

In [3]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

NameError: name 'train_ds' is not defined

In [25]:
num_classes = 5

model = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [26]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [17]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [22]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=3
)

Epoch 1/3


ResourceExhaustedError:  OOM when allocating tensor with shape[32,32,178,178] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node sequential/conv2d/Relu (defined at <ipython-input-16-ce06c3301011>:4) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_1178]

Function call stack:
train_function


In [ ]:
model.save('./my_model_01.h5')

In [24]:

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 24,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('test_set',
                                            target_size = (64, 64),
                                            batch_size = 24,
                                            class_mode = 'binary')


Found 2572 images belonging to 3 classes.
Found 400 images belonging to 3 classes.


In [ ]:
# 추가
import os
MODEL_DIR = './model'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)
    
modelpath="./model/{epoch:02d}-{val_loss:.4f}.hdf5"

checkpointer = ModelCheckpoint(filepath = modelpath, monitor='val_loss', verbose=1, save_best_only=True)

In [45]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                   save_weights_only=True,
                                   verbose=1)# Train the model with the new callback

In [46]:
model.fit(
    training_set,
    validation_data=test_set,
    epochs=3,
    callbacks=[cp_callback]
)

Epoch 1/3
108/108 [==============================] - ETA: 0s - loss: 1.0178 - accuracy: 0.4510
Epoch 00001: saving model to training_1/cp.ckpt
108/108 [==============================] - 111s 1s/step - loss: 1.0178 - accuracy: 0.4510 - val_loss: 0.9024 - val_accuracy: 0.6500
Epoch 2/3
108/108 [==============================] - ETA: 0s - loss: 1.0090 - accuracy: 0.4510
Epoch 00002: saving model to training_1/cp.ckpt
108/108 [==============================] - 108s 1s/step - loss: 1.0090 - accuracy: 0.4510 - val_loss: 0.9625 - val_accuracy: 0.6500
Epoch 3/3
108/108 [==============================] - ETA: 0s - loss: 1.0068 - accuracy: 0.4545
Epoch 00003: saving model to training_1/cp.ckpt
108/108 [==============================] - 108s 1000ms/step - loss: 1.0068 - accuracy: 0.4545 - val_loss: 0.9540 - val_accuracy: 0.6500


In [54]:
ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


In [47]:
print("-- Evaluate --")
scores = model.evaluate_generator(test_set, steps=5)

#print(his.history['loss'])
#print(his.history['acc'])
#print(his.history['val_loss'])
#print(his.history['val_acc'])

#classifier.save("face_shape_model.h5")

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

-- Evaluate --
accuracy: 67.50%


In [55]:
model.save('models/my_model_02')

INFO:tensorflow:Assets written to: models/my_model_02/assets


In [52]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

In [35]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_2 (Rescaling)      (None, None, None, None)  0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, None, None, 32)    896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, None, None, 32)    0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, None, None, 32)    9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, None, None, 32)    0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, None, None, 32)    9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, None, None, 32)   

In [56]:
new_model = tf.keras.models.load_model('models/my_model_02')

In [57]:
# 모델 구조를 확인합니다
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_2 (Rescaling)      (None, None, None, None)  0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, None, None, 32)    896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, None, None, 32)    0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, None, None, 32)    9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, None, None, 32)    0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, None, None, 32)    9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, None, None, 32)   

In [77]:
print("-- Evaluate --")
loss, scores = new_model.evaluate(test_set, steps=5)

#print(his.history['loss'])
#print(his.history['acc'])
#print(his.history['val_loss'])
#print(his.history['val_acc'])

#classifier.save("face_shape_model.h5")
print('복원된 모델의 정확도: {:5.2f}%'.format(100*scores))

-- Evaluate --
5/5 [==============================] - 3s 686ms/step - loss: 0.9705 - accuracy: 0.0000e+00
복원된 모델의 정확도:  0.00%


0.9705077409744263

In [84]:
from tensorflow.keras.preprocessing import image
test_image = image.load_img('./test_image/ex2.jpg', target_size = (64, 64))
#test_image = image.img_to_array(test_image)
#test_image = np.expand_dims(test_image, axis = 0)
#output = classifier.predict_generator(test_image, steps=5)
#output
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

In [85]:
result = model.predict(test_image)

In [86]:
np.around(result)
result=result.argmax()
print(result)

1


In [87]:
training_set.class_indices

{'Oblong': 0, 'Round': 1, 'Square': 2}

In [88]:
if result == 0:
    prediction = 'Oblong'
elif result == 1:
    prediction = 'Round'
else:
    prediction = 'Square'

In [ ]:
prediction